<a href="https://colab.research.google.com/github/dinhluan14/Artificial-Intelligence/blob/main/B%E1%BA%A3n_sao_c%E1%BB%A7a_XLM_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!nvidia-smi

Sat Sep 18 05:07:04 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [61]:
MAX_LEN = 10 # This value could be set as 256, 512 etc.

sentence1 = 'Hello there.'
sentence2 = 'i am thankyou'

encoded_dict = tokenizer.encode_plus(
            sentence1,sentence2 ,                     # Sentence to encode.
            add_special_tokens = True, 
            truction=True ,    # Add '[CLS]' and '[SEP]'
            max_length =20 ,           # Pad or truncate.
            pad_to_max_length = True,
            return_attention_mask = True,   # Construct attn. masks.
            return_tensors = 'pt',          # Return pytorch tensors.
           )


encoded_dict

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Keyword arguments {'truction': True} not recognized.
Keyword arguments {'truction': True} not recognized.


{'input_ids': tensor([[    0, 31414,    89,     4,     2,     2,   118,   524,    87,  4122,
          1438,     2,     1,     1,     1,     1,     1,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])}

In [160]:
padded_token_list = encoded_dict['input_ids'][0]

In [161]:
padded_token_list

tensor([    0, 31414,    89,     4,     2,     2,   118,   524,    87,  4122,
         1438,     2,     1,     1,     1,     1,     1,     1,     1,     1])

In [62]:
premise_id=tokenizer.encode(sentence1, add_special_tokens = False)
hypothesis_id=tokenizer.encode(sentence2, add_special_tokens = False)
premise_len = len(premise_id)
hypothesis_len = len(hypothesis_id)

segment_ids = torch.tensor([0] * (premise_len + 2) + [1] * (hypothesis_len + 1))  # sentence 0 and sentence 1
attention_mask_ids = torch.tensor([1] * (premise_len + hypothesis_len + 3+2))

In [199]:
attention_mask_ids

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [63]:
pair_token_ids = [tokenizer.cls_token_id] + premise_id + [tokenizer.sep_token_id] + hypothesis_id + [tokenizer.sep_token_id]+[1,1]

In [66]:
pair_token_ids

[0, 31414, 89, 4, 2, 118, 524, 87, 4122, 1438, 2, 1, 1]

In [88]:
x=10-len(pair_token_ids)

In [89]:
for i in range(-x):
  pair_token_ids.pop(12-i)


In [90]:
pair_token_ids

[0, 31414, 89, 4, 2, 118, 524, 87, 4122, 1438]

In [195]:
len(pair_token_ids)
x=[]
for i in range(13):
  x.append(1)

In [196]:
x

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [171]:
token_ids=[]
token_ids.append(torch.tensor(pair_token_ids))

In [173]:
torch.tensor(pair_token_ids)

tensor([    0, 31414,    89,     4,     2,   118,   524,    87,  4122,  1438,
            2])

In [172]:
token_ids

[tensor([    0, 31414,    89,     4,     2,   118,   524,    87,  4122,  1438,
             2])]

In [ ]:
from torch.nn.utils.rnn import pad_sequence
token_ids = pad_sequence(token_ids, batch_first=True)

In [165]:
token_ids[0]

tensor([    0, 31414,    89,     4,     2,   118,   524,    87,  4122,  1438,
            2])

In [159]:
token_ids

tensor([[    0, 31414,    89,     4,     2,  9178,    32,    47,     2]])

In [118]:
attention_mask_ids

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1])

In [6]:
!pip install transformers

     |████████████████████████████████| 2.8 MB 9.0 MB/s 
     |████████████████████████████████| 895 kB 59.2 MB/s 
     |████████████████████████████████| 636 kB 61.2 MB/s 
     |████████████████████████████████| 52 kB 1.5 MB/s 
     |████████████████████████████████| 3.3 MB 36.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [7]:
!pip install sentencepiece

     |████████████████████████████████| 1.2 MB 9.9 MB/s 


In [8]:
import pandas as pd
import numpy as np
import os
import gc

import random

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# set a seed value
torch.manual_seed(555)

from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score, accuracy_score

import transformers
from transformers import BertTokenizer, BertForSequenceClassification 
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification
from transformers import AdamW

import warnings
warnings.filterwarnings("ignore")

In [9]:
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification
from transformers import RobertaTokenizer, RobertaModel
MODEL_TYPE = 'roberta-base'

# tokenizer = XLMRobertaTokenizer.from_pretrained(MODEL_TYPE)
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
!pip install datasets

     |████████████████████████████████| 270 kB 8.0 MB/s 
     |████████████████████████████████| 243 kB 45.9 MB/s 
     |████████████████████████████████| 119 kB 50.5 MB/s 
     |████████████████████████████████| 1.3 MB 36.6 MB/s 
     |████████████████████████████████| 294 kB 49.4 MB/s 
     |████████████████████████████████| 142 kB 56.3 MB/s 


In [11]:
from datasets import load_dataset
datasets = load_dataset('multi_nli')

Downloading:   0%|          | 0.00/1.90k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

Using custom data configuration default


Downloading:   0%|          | 0.00/227M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset multi_nli downloaded and prepared to /root/.cache/huggingface/datasets/multi_nli/default/0.0.0/591f72eb6263d1ab527561777936b199b714cda156d35716881158a2bd144f39. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [113]:
df_train['sentances1'][15000]

"6. Boys Don't Cry . Starkly beautiful, Kimberly Peirce's debut film has at its core a tragicomic  That the cross-dressing Brandon Teena, a k a Teena Brandon (the rapturous Hillary Swank) feels most at home among the sort of roughnecks who would kill her if they knew her true gender."

In [114]:
df_train['sentances2'][15000]

"6. Boys Don't Cry . Starkly beautiful, Kimberly Peirce's debut film has at its core a tragicomic  That the cross-dressing Brandon Teena, a k a Teena Brandon (the rapturous Hillary Swank) feels most at home among the sort of roughnecks who would kill her if they knew her true gender."

In [12]:
sentances1=datasets['train']['premise']
sentances2=datasets['train']['hypothesis']
lable=datasets["train"]['label']
sentances1_vla=datasets['validation_matched']['premise']
sentances2_vla=datasets['validation_matched']['hypothesis']
lable_vla=datasets["validation_matched"]['label']
data1={ "sentances1":sentances1,
       "sentances2":sentances1,
       "lable":lable

}
data2={ "sentances1":sentances1_vla,
       "sentances2":sentances1_vla,
       "lable":lable_vla

}

In [48]:
data={ "sentances1":"Hello there.",
       "sentances2":'How are you?',
      "lable":0
       

}
index=1
test=pd.Series(data).to_frame()

In [49]:
test=test.T
df_test = test.reset_index(drop=True)

In [13]:
import pandas as pd
train=pd.DataFrame(data=data1)
vla=pd.DataFrame(data=data2)
df_train = train.reset_index(drop=True)
df_val = vla.reset_index(drop=True)

In [70]:
--3

3

In [97]:
import torch
class CompDataset(torch.utils.data.Dataset):

    def __init__(self, df):
        self.df_data = df
        self.tokenizer=RobertaTokenizer.from_pretrained("roberta-base")



    def __getitem__(self, index):
        sentence1 = self.df_data.loc[index, 'sentances1']
        sentence2 = self.df_data.loc[index, 'sentances2']
        
        y=[]

        premise_id = self.tokenizer.encode(sentence1, add_special_tokens = False)
        hypothesis_id = self.tokenizer.encode(sentence2, add_special_tokens = False)
        pair_token_ids = [self.tokenizer.cls_token_id] + premise_id + [self.tokenizer.sep_token_id] + hypothesis_id + [self.tokenizer.sep_token_id]
        a=len(pair_token_ids)
        x=256-len(pair_token_ids)
        if x>=0:
          for i in range(x):
            y.append(1)
          pair_token_ids = [self.tokenizer.cls_token_id] + premise_id + [self.tokenizer.sep_token_id] + hypothesis_id + [self.tokenizer.sep_token_id]+y
        if x<0:
          for i in range(-x):
            pair_token_ids.pop(a-1-i)

        premise_len = len(premise_id)
        hypothesis_len = len(hypothesis_id)
        attention_mask_ids = torch.tensor([1] * (256))  # mask padded values

        token_ids=torch.tensor(pair_token_ids)
        mask_ids=attention_mask_ids
        
        
        
        target = torch.tensor(self.df_data.loc[index, 'lable'])

        sample = (token_ids, mask_ids,target)


        return sample


    def __len__(self):
        return len(self.df_data)

In [72]:
test_data = CompDataset(df_test)

In [57]:
torch.tensor([1] *10)

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [73]:
test_dataloader = torch.utils.data.DataLoader(test_data,
                                        batch_size=1,
                                        shuffle=True,
                                       num_workers=2)

In [74]:
batch = next(iter(test_dataloader))

# Send the data to the device
b_input_ids = batch[0]
b_input_mask = batch[1]
b_labels = batch[2]


In [75]:
b_input_ids.shape

torch.Size([1, 256])

In [55]:
b_input_mask

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

In [183]:
b_input_ids

tensor([[    0, 31414,    89,     4,     2,  6179,    32,    47,   116,     2]])

In [181]:
b_input_mask

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

In [38]:
print('bos_token_id <s>:', tokenizer.bos_token_id)
print('eos_token_id </s>:', tokenizer.eos_token_id)
print('sep_token_id </s>:', tokenizer.sep_token_id)
print('pad_token_id <pad>:', tokenizer.pad_token_id)


bos_token_id <s>: 0
eos_token_id </s>: 2
sep_token_id </s>: 2
pad_token_id <pad>: 1


In [35]:
b_input_ids

tensor([[    0, 31414,    89,     2,     2,  6179,    32,     2]])

In [36]:
MAX_LEN = 15

sentence1 = 'Hello there.'
sentence2 = 'How are you?'

encoded_dict = tokenizer.encode_plus(
            sentence1, sentence2,           # Sentences to encode.
            add_special_tokens = True,      # Add '[CLS]' and '[SEP]'
            max_length = 8,           # Pad or truncate.
            pad_to_max_length = True,
            return_attention_mask = True,   # Construct attn. masks.
            return_tensors = 'pt',          # Return pytorch tensors.
           )


encoded_dict

{'input_ids': tensor([[    0, 31414,    89,     2,     2,  6179,    32,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}

In [98]:
train_data = CompDataset(df_train)
val_data = CompDataset(df_val)


In [99]:
train_dataloader = torch.utils.data.DataLoader(train_data,
                                        batch_size=32,
                                        shuffle=True,
                                       num_workers=2)
val_dataloader = torch.utils.data.DataLoader(val_data,
                                        batch_size=32,
                                        shuffle=True,
                                       num_workers=2)


In [100]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [78]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=30_522,  # we align this to the tokenizer vocab_size
    max_position_embeddings=514,
    hidden_size=768,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1
)

In [79]:
from transformers import RobertaForSequenceClassification
MODEL_TYPE="roberta-base"
# tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
model = RobertaForSequenceClassification.from_pretrained(
    MODEL_TYPE, 
    num_labels = 3,
)


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.bias', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

In [83]:
model.to(device)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

In [80]:
batch = next(iter(train_dataloader))

# Send the data to the device
b_input_ids = batch[0].to(device)
b_input_mask = batch[1].to(device)
b_labels = batch[2].to(device)

# Run the model
outputs = model(b_input_ids, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)

# The ouput is a tuple (loss, preds).
outputs

SequenceClassifierOutput([('loss',
                           tensor(1.1200, device='cuda:0', grad_fn=<NllLossBackward>)),
                          ('logits', tensor([[ 0.0118, -0.0155, -0.0046],
                                   [-0.1658,  0.0259, -0.1306],
                                   [ 0.0884,  0.1279, -0.2657],
                                   [ 0.0204,  0.0280, -0.0153],
                                   [-0.3118,  0.2087, -0.2475],
                                   [-0.1436,  0.1099, -0.0398],
                                   [-0.2281,  0.1986, -0.1945],
                                   [-0.0957,  0.1301, -0.2161],
                                   [-0.1804,  0.1489, -0.1194],
                                   [-0.1174,  0.1169, -0.0529],
                                   [-0.1919,  0.0721, -0.2892],
                                   [-0.0715,  0.0563, -0.1233],
                                   [ 0.0141,  0.0226, -0.0158],
                                   

In [ ]:
len(b_input_ids)

8

In [94]:
from transformers import  AdamW
optimizer = AdamW(model.parameters(), lr=2e-5)

In [95]:
def multi_acc(y_pred, y_test):
  acc = (torch.log_softmax(y_pred, dim=1).argmax(dim=1) == y_test).sum().float() / float(y_test.size(0))
  return acc

import time

EPOCHS = 5

def train(model, train_loader, val_loader, optimizer):  
  total_step = len(train_loader)

  for epoch in range(EPOCHS):
    start = time.time()
    model.train()
    total_train_loss = 0
    total_train_acc  = 0
    for batch_idx, (b_input_ids,b_input_mask, y) in enumerate(train_loader):
      optimizer.zero_grad()
      b_input_ids = b_input_ids.to(device)
      b_input_mask = b_input_mask.to(device)
      labels = y.to(device)

      loss, prediction = outputs = model(b_input_ids, 
                        attention_mask=b_input_mask, 
                        labels=labels).values()

      acc = multi_acc(prediction, labels)

      loss.backward()
      optimizer.step()
      
      total_train_loss += loss.item()
      total_train_acc  += acc.item()

    train_acc  = total_train_acc/len(train_loader)
    train_loss = total_train_loss/len(train_loader)
    model.eval()
    total_val_acc  = 0
    total_val_loss = 0
    with torch.no_grad():
      for batch_idx, (b_input_ids,b_input_mask, y) in enumerate(val_loader):
        b_input_ids = b_input_ids.to(device)
        b_input_mask = b_input_mask.to(device)
        labels = y.to(device)

        loss, prediction = outputs = model(b_input_ids, 
                        attention_mask=b_input_mask, 
                        labels=labels).values()
        
        
        
        acc = multi_acc(prediction, labels)

        total_val_loss += loss.item()
        total_val_acc  += acc.item()

    val_acc  = total_val_acc/len(val_loader)
    val_loss = total_val_loss/len(val_loader)
    end = time.time()
    hours, rem = divmod(end-start, 3600)
    minutes, seconds = divmod(rem, 60)

    print(f'Epoch {epoch+1}: train_loss: {train_loss:.4f} train_acc: {train_acc:.4f} | val_loss: {val_loss:.4f} val_acc: {val_acc:.4f}')
    print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

In [101]:
train(model, train_dataloader, val_dataloader, optimizer)

Epoch 1: train_loss: 1.0998 train_acc: 0.3338 | val_loss: 1.0981 val_acc: 0.3275
02:38:23.10


KeyboardInterrupt: ignored